http://www.dcfnerds.com/analyzing-technical-analysis/

In [1]:
import datetime as dt
import pandas as pd
from pandas_datareader import data
import cPickle as pickle

ImportError: No module named 'pandas_datareader'

In [2]:
universe = pd.read_csv('inputs/universe.csv', index_col='symbol')
universe.index = map(lambda x: x.replace('.', '-'), universe.index)

FileNotFoundError: File b'inputs/universe.csv' does not exist

In [3]:
shortUniverse = True

In [4]:
def pull_universe(uni, sdate, edate):
    '''Expects a universe csv. Returns a dataframe of prices.
    
    Might not work if an error is generated on the first stock in the index.
    Performance and stability could be improved through list comprehension
    and a single pd.concat operation.'''
    
    # Start by pulling one stock.
    df = data.DataReader(uni.index[0],data_source='yahoo', start=sdate, end=edate)
    df['symbol'] = uni.index[0]
    df['sector'] = uni.loc[uni.index[0],'sector']
    
    #Loop through remaining stocks. Append to prices file.
    for Tic in uni.index[1:]:
        try:
            df_new = data.DataReader(Tic,data_source='yahoo', start=sdate, end=edate)
            df_new['symbol'] = Tic
            df_new['sector'] = uni.loc[Tic,'sector']
            df = pd.concat([df, df_new])
        except IOError:
            pass

    return df

In [6]:
if shortUniverse:
    prices = pull_universe(uni=universe.iloc[:3, ], sdate=dt.datetime(2015, 1, 1), edate=dt.datetime(2015, 12, 31))
else:
    prices = pull_universe(uni=universe, sdate=dt.datetime(2000, 1, 1), edate=dt.datetime(2015, 12, 31))

NameError: name 'universe' is not defined

In [7]:
prices.head()

NameError: name 'prices' is not defined

In [8]:
if shortUniverse:
    with open('intermediaries/pricesRaw.p', 'wb') as handle:
        pickle.dump(prices, handle)
else:
    with open('intermediaries/pricesRaw-full.p', 'wb') as handle:
        pickle.dump(prices, handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/pricesRaw.p'

In [9]:
prices['last_volume'] = prices.groupby('symbol')['Volume'].shift(1)
prices.loc[prices['symbol']==universe.index[7], :].head()

NameError: name 'prices' is not defined

In [10]:
import numpy as np
import cPickle as pickle
shortUniverse = False

ImportError: No module named 'cPickle'

In [11]:
if shortUniverse:
    with open('intermediaries/pricesRaw.p', 'rb') as handle:
        prices = pickle.load(handle)
else:
    with open('intermediaries/pricesRaw-full.p', 'rb') as handle:
        prices = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/pricesRaw.p'

In [12]:
def adjust_prices(df):
    '''Adjusts df prices for dividends and splits.'''
    df['AdjFactor'] = df.groupby('symbol')['Adj Close'].shift(1) / df.groupby('symbol')['Close'].shift(1)
    df['Open'] = df['Open'] * df['AdjFactor']
    df['High'] = df['High'] * df['AdjFactor']
    df['Low'] = df['Low'] * df['AdjFactor']
    df['Close'] = df['Close'] * df['AdjFactor']
    new_df = df[['sector', 'symbol', 'Open', 'High', 'Low', 'Close', 'Volume']]
    new_df.columns = ['sector', 'symbol', 'open', 'high', 'low', 'close', 'volume']
    return new_df

In [13]:
prices = adjust_prices(prices)

NameError: name 'prices' is not defined

In [14]:
def add_returns(df):
    df['lastClose'] = df.groupby('symbol')['close'].shift(1)
    df['ret_cc'] = np.log(df['close'] / df['lastClose'])
    df['ret_oc'] = np.log(df['close'] / df['open'])
    df['ret_co'] = np.log(df['open'] / df['lastClose'])
    df.drop('lastClose', 1, inplace=True)
    return df

In [15]:
prices = add_returns(prices)

NameError: name 'prices' is not defined

In [16]:
prices.head(20)

NameError: name 'prices' is not defined

In [17]:
if shortUniverse:
    with open('intermediaries/prices.p', 'wb') as handle:
        pickle.dump(prices, handle)
else:
    with open('intermediaries/prices-full.p', 'wb') as handle:
        pickle.dump(prices, handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/prices.p'

In [18]:
import numpy as np
import pandas as pd
import cPickle as pickle
shortUniverse = True

ImportError: No module named 'cPickle'

In [19]:
if shortUniverse:
    with open('intermediaries/prices.p', 'rb') as handle:
        prices = pickle.load(handle)
else:
    with open('intermediaries/prices-full.p', 'rb') as handle:
        prices = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/prices.p'

In [20]:
def add_MA(df, lag):
    df['ma_'+str(lag)]= df.groupby('symbol')['close'].apply(pd.rolling_mean, lag)
    df['ma_'+str(lag)+'_lag_1']= df.groupby('symbol')['ma_'+str(lag)].shift(1)
    return df

In [21]:
prices = add_MA(prices, lag=5)

NameError: name 'prices' is not defined

In [22]:
def add_VWMA(df, lag):
    df['typical'] = (df['low'] + df['high'] + df['close']) / 3
    df['VP'] = df['volume'] * df['typical']
    df['VPLag' + str(lag)]= df.groupby('symbol')['VP'].apply(pd.rolling_mean, lag)
    df['VLag' + str(lag)]= df.groupby('symbol')['volume'].apply(pd.rolling_mean, lag)
    df['vwma_' + str(lag)] = df['VPLag' + str(lag)] / df['VLag' + str(lag)]
    df['vwma_' + str(lag) + '_lag_1'] = df.groupby('symbol')['vwma_' + str(lag)].shift(1)
    df = df.drop(['typical', 'VP', 'VPLag' + str(lag), 'VLag' + str(lag)], axis=1)
    return df

In [23]:
prices = add_VWMA(prices, lag=5)

NameError: name 'prices' is not defined

In [24]:
prices.head(10)

NameError: name 'prices' is not defined

In [25]:
import numpy as np
import pandas as pd
import cPickle as pickle
shortUniverse = True

ImportError: No module named 'cPickle'

In [26]:
if shortUniverse:
    with open('intermediaries/prices.p', 'rb') as handle:
        prices = pickle.load(handle)
else:
    with open('intermediaries/prices-full.p', 'rb') as handle:
        prices = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/prices.p'

In [27]:
def add_RSI(df, lag=14):
    df['U'] = df['close'] - df.groupby('symbol')['close'].shift(1)
    df['D'] = -1 * df['U']
    df.loc[df['U'] < 0, 'U'] = 0
    df.loc[df['D'] < 0, 'D'] = 0
    df['U_EMA'] = pd.ewma(df['U'], span=14, min_periods=14, adjust=True)
    df['D_EMA'] = pd.ewma(df['D'], span=14, min_periods=14, adjust=True)
    df.loc[df['D_EMA'] == 0, 'D_EMA'] = .01
    df['RS'] = df['U_EMA'] / df['D_EMA']
    df['RSI'] = 100 - (100 / (1 + df['RS']))
    df = df.drop(['U', 'D', 'U_EMA', 'D_EMA', 'RS'], axis=1)
    return df

In [28]:
prices = add_RSI(prices)

NameError: name 'prices' is not defined

In [29]:
prices.head(20).tail(10)

NameError: name 'prices' is not defined

In [30]:
import numpy as np
import pandas as pd
import cPickle as pickle
shortUniverse = True

ImportError: No module named 'cPickle'

In [31]:
if shortUniverse:
    with open('intermediaries/prices.p', 'rb') as handle:
        prices = pickle.load(handle)
else:
    with open('intermediaries/prices-full.p', 'rb') as handle:
        prices = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/prices.p'

In [32]:
def add_EoM(df, lag=14):
    df['medPrice'] = (df['high'] + df['low'])/2
    df['dist'] = df['medPrice'] - df.groupby('symbol')['medPrice'].shift(1)
    df['box'] = (df['volume']/1e8)/(df['high'] - df['low'])
    df['EV1'] = df['dist'] / df['box']
    df['EoM'] = df.groupby('symbol')['EV1'].apply(pd.rolling_mean, lag)
    df = df.drop(['medPrice', 'dist', 'box', 'EV1'], axis=1)
    return df

In [33]:
prices = add_EoM(prices)

NameError: name 'prices' is not defined

In [34]:
prices.head(20).tail(10)

NameError: name 'prices' is not defined

In [35]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import math
import helper_functions as hf
%matplotlib inline
from scipy import stats
import cPickle as pickle

shortUniverse = True

ImportError: No module named 'seaborn'

In [36]:
if shortUniverse:
    with open('intermediaries/prices.p', 'rb') as handle:
        prices = pickle.load(handle)
else:
    with open('intermediaries/prices-full.p', 'rb') as handle:
        prices = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/prices.p'

In [37]:
def add_fake_signals(df):
    df['rand'] = np.random.uniform(size=len(df.index))
    df['fake_signal'] = 'neut'
    df.loc[df['rand'] <= .2, 'fake_signal'] = 'bull'
    df.loc[df['rand'] >= .9, 'fake_signal'] = 'bear'
    df.drop('rand', axis=1, inplace=True)
    return df

In [38]:
prices = add_fake_signals(prices)

NameError: name 'prices' is not defined

In [39]:
def add_age_num(df, cType, strat):
    """Adds the age of a given fake_signal.
    
    cType should be either 'bull' or 'bear'.
    strat should be the name of a column of fake_signals with 'bull' and 'bear' values.
    Can be improved through parallel functions. This is an expensive function to run.
    """
    if cType == 'bull':
        same, diff = 'bull', 'bear'
    else:
        same, diff = 'bear', 'bull'

    df[strat + '_age_' + cType] = np.nan
    for i in range(1, len(df[strat + '_age_' + cType]) - 1):
    	# if new stock, then age = 0
        if df.ix[i - 1, 'symbol'] != df.ix[i, 'symbol']:
            df.ix[i, strat + '_age_' + cType] = 0
        # if last fake_signal was same, age = 1
        elif df.ix[i - 1, strat] == same:
            df.ix[i, strat + '_age_' + cType] = 1
        # if last fake_signal was diff, age = 0
        elif df.ix[i - 1, strat] == diff:
            df.ix[i, strat + '_age_' + cType] = 0
        # if last fake_signal was neut and last age >0, age=last age + 1
        elif ((df.ix[i - 1, strat] == 'neut') & (df.ix[i - 1, strat + '_age_' + cType] > 0)):
            df.ix[i, strat + '_age_' + cType] = df.ix[i - 1, strat + '_age_' + cType] + 1
        # if last fake_signal was neut and last age == 0, age = 0
        elif ((df.ix[i - 1, strat] == 'neut') & (df.ix[i - 1, strat + '_age_' + cType] == 0)):
            df.ix[i, strat + '_age_' + cType] = 0
        # This should be triggered near the beginning when we haven't seen anything yet.
        else:
            df.ix[i, strat + '_age_' + cType] = 0

    return df

def add_age_nums(df, strat):
    """Wrapper function to generate both bull and bear signals.
    
    This function is currently iterative and can take a long time to run."""
    df = add_age_num(df, 'bull', strat)
    df = add_age_num(df, 'bear', strat)
    return df

In [40]:
prices = add_age_nums(prices, strat='fake_signal')

NameError: name 'prices' is not defined

In [41]:
# http://libguides.library.kent.edu/SPSS/OneSampletTest

def get_results_table(df, strat, lag):
    resultsBull = df[[strat + '_age_' + 'bull', 'ret_cc']].groupby(strat + '_age_' + 'bull').describe().unstack(1)
    resultsBull = resultsBull[0:lag]['ret_cc']
    resultsBull['testStat'] = resultsBull['mean'] / resultsBull['std'] / np.sqrt(resultsBull['count'])
    resultsBull['p-value'] = stats.t.sf(np.abs(resultsBull['testStat']), resultsBull['count']-1)*2
    resultsBull.drop(['25%', '75%'], axis=1, inplace=True)
    
    resultsBear = df[[strat + '_age_' + 'bear', 'ret_cc']].groupby(strat + '_age_' + 'bear').describe().unstack(1)
    resultsBear = resultsBear[0:lag]['ret_cc']
    resultsBear['testStat'] = resultsBear['mean'] / resultsBear['std'] / np.sqrt(resultsBear['count'])
    resultsBear['p-value'] = stats.t.sf(np.abs(resultsBear['testStat']), resultsBear['count']-1)*2
    resultsBear.drop(['25%', '75%'], axis=1, inplace=True)
    
    results = pd.concat({'bull':resultsBull, 'bear':resultsBear}, axis=1)
    results.index.name = 'signal_age'
    return results

In [42]:
def strat_results(df, strat, lag):
    '''Returns a results table for a given strategy for a given number of lags.
    
    The prices table fed into df must contain a column strat.
    Column strat should contain "bull" and "bear" values.'''
    df = add_age_nums(df, strat)
    results = get_results_table(df, strat, lag)
    
    return results

results_fake_signal = strat_results(prices, 'fake_signal', 5)
results_fake_signal

NameError: name 'prices' is not defined

In [43]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import math
import helper_functions as hf
%matplotlib inline
from scipy import stats
import cPickle as pickle

shortUniverse = True
loadResults = False

ImportError: No module named 'seaborn'

In [44]:
if shortUniverse:
    with open('intermediaries/prices.p', 'rb') as handle:
        prices = pickle.load(handle)
else:
    with open('intermediaries/prices-full.p', 'rb') as handle:
        prices = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'intermediaries/prices.p'

In [45]:
def add_ma_15_30_cross(df):
    df = hf.add_MA(df, lag=15)
    df = hf.add_MA(df, lag=30)
    
    df['ma15_ma30_cross'] = 'neut'
    df.loc[((df['ma_15_lag_1'] < df['ma_30_lag_1']) & (df['ma_15'] > df['ma_30'])),'ma15_ma30_cross'] = 'bull'
    df.loc[((df['ma_15_lag_1'] > df['ma_30_lag_1']) & (df['ma_15'] < df['ma_30'])),'ma15_ma30_cross'] = 'bear'
    df.drop(['ma_15', 'ma_15_lag_1', 'ma_30', 'ma_30_lag_1'], axis=1, inplace=True)
    return df

results_ma_15_30_cross = hf.strat_results(add_ma_15_30_cross(prices),
                                          strat='ma15_ma30_cross',
                                          lag=5)
results_ma_15_30_cross

NameError: name 'hf' is not defined

In [46]:
def add_ma_10_20_cross(df):
    df = hf.add_MA(df, lag=10)
    df = hf.add_MA(df, lag=20)
    
    df['ma10_ma20_cross'] = 'neut'
    df.loc[((df['ma_10_lag_1'] < df['ma_20_lag_1']) & (df['ma_10'] > df['ma_20'])),'ma10_ma20_cross'] = 'bull'
    df.loc[((df['ma_10_lag_1'] > df['ma_20_lag_1']) & (df['ma_10'] < df['ma_20'])),'ma10_ma20_cross'] = 'bear'
    df.drop(['ma_10', 'ma_10_lag_1', 'ma_20', 'ma_20_lag_1'], axis=1, inplace=True)
    return df

results_ma_10_20_cross = hf.strat_results(add_ma_10_20_cross(prices),
                                          strat='ma15_ma30_cross',
                                          lag=5)
results_ma_10_20_cross

NameError: name 'hf' is not defined

In [47]:
def add_ma_50_200_cross(df):
    df = hf.add_MA(df, lag=50)
    df = hf.add_MA(df, lag=200)
    
    df['ma50_ma200_cross'] = 'neut'
    df.loc[((df['ma_50_lag_1'] < df['ma_200_lag_1']) & (df['ma_50'] > df['ma_200'])),'ma50_ma200_cross'] = 'bull'
    df.loc[((df['ma_50_lag_1'] > df['ma_200_lag_1']) & (df['ma_50'] < df['ma_200'])),'ma50_ma200_cross'] = 'bear'
    df.drop(['ma_50', 'ma_50_lag_1', 'ma_200', 'ma_200_lag_1'], axis=1, inplace=True)
    return df

results_ma_50_200_cross = hf.strat_results(add_ma_50_200_cross(prices),
                                          strat='ma15_ma30_cross',
                                          lag=5)
results_ma_50_200_cross

NameError: name 'hf' is not defined

In [48]:
def add_ma_bounce_10(df):
    df = hf.add_MA(df, lag=10)
    
    df['ma_bounce_10'] = 'neut'
    df.loc[(df['ma_10'] < df['open']) & (df['ma_10'] > df['low']) & (df['ma_10'] < df['close']),'ma_bounce_10'] = 'bull'
    df.loc[(df['ma_10'] > df['open']) & (df['ma_10'] < df['high']) & (df['ma_10'] > df['close']),'ma_bounce_10'] = 'bear'
    df.drop(['ma_10', 'ma_10_lag_1'], axis=1, inplace=True)
    return df

results_ma_bounce_10 = hf.strat_results(add_ma_bounce_10(prices),
                                          strat='ma_bounce_10',
                                          lag=5)
results_ma_bounce_10

NameError: name 'hf' is not defined

In [49]:
def add_ma_bounce_30(df):
    df = hf.add_MA(df, lag=30)
    
    df['ma_bounce_30'] = 'neut'
    df.loc[(df['ma_30'] < df['open']) & (df['ma_30'] > df['low']) & (df['ma_30'] < df['close']),'ma_bounce_30'] = 'bull'
    df.loc[(df['ma_30'] > df['open']) & (df['ma_30'] < df['high']) & (df['ma_30'] > df['close']),'ma_bounce_30'] = 'bear'
    df.drop(['ma_30', 'ma_30_lag_1'], axis=1, inplace=True)
    return df

results_ma_bounce_30 = hf.strat_results(add_ma_bounce_30(prices),
                                          strat='ma_bounce_30',
                                          lag=5)
results_ma_bounce_30

NameError: name 'hf' is not defined

In [50]:
def add_ma_bounce_200(df):
    df = hf.add_MA(df, lag=200)
    
    df['ma_bounce_200'] = 'neut'
    df.loc[(df['ma_200'] < df['open']) & (df['ma_200'] > df['low']) & (df['ma_200'] < df['close']),'ma_bounce_200'] = 'bull'
    df.loc[(df['ma_200'] > df['open']) & (df['ma_200'] < df['high']) & (df['ma_200'] > df['close']),'ma_bounce_200'] = 'bear'
    df.drop(['ma_200', 'ma_200_lag_1'], axis=1, inplace=True)
    return df

results_ma_bounce_200 = hf.strat_results(add_ma_bounce_200(prices),
                                          strat='ma_bounce_200',
                                          lag=5)
results_ma_bounce_200

NameError: name 'hf' is not defined